In [ ]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [ ]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [ ]:
 # Examine the results, then determine element that contains sought info
print(soup.prettify())

In [ ]:
 # Print all paragraph texts
titles = soup.find_all('div', class_='content_title')
for article_title in titles:
    print(article_title.text)

In [ ]:
# find the latest title and paragraph then store in variables

news_title = soup.find("div", class_="content_title").text
news_p = soup.find("div", class_="rollover_description_inner").text

print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

In [ ]:
# Set the directory where chromedriver exists
executable_path = {'executable_path':'C:/bin/chromedriver.exe'} 
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Visit JPL Mars Space Images for JPL Featured Space Image
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
Browser.visit(url_image)
#Browser.click_link_by_partial_text('full_image')

In [ ]:
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
new_html = browser.html
new_soup = bs(new_html, 'html.parser')
temp_img_url = new_soup.find('img', class_='fancybox_image')
back_half_img_url = temp_img_url.get('src')

featured_mars_image_url = "https://www.jpl.nasa.gov" + back_half_img_url

print(featured_mars_image_url)